In [1]:
# IPYTHON_TEST_SKIP_REMAINDER
#
# This flag causes execution to skip the remainder of the notebook in the Jenkins test suite.
# We do this until we have found a way to make it work reliably there.

# tutorial-mayavi-oop-animation

#### Author: Mark Vousden

#### Last Modified: 28/03/2014

This notebook is aimed at users of finmag who have looked at the previous Mayavi tutorial (tutorial-visualising-magnetisation-using-mayavi-mlab) and want to gain familiarity with the OOP approach of using Mayavi. In this notebook we will again create sample data, but this time we will write VTU files representing the DMI energy density as a skyrmion relaxes. These will be loaded by Mayavi to draw surface plots of the DMI energy density of the system as it relaxes. The background colour will be changed and these drawings will be used to generate an animation. As with before, much of the information here has been obtained from the Mayavi documentation[1].

In [1]:
import dolfin as df
import finmag
import mayavi
import mayavi.api
import mayavi.modules.surface
import mayavi.filters.extract_vector_components 
import numpy as np
import os

[2014-06-09 22:37:08] INFO: Finmag logging output will be appended to file: '/home/albert/.finmag/global.log'
[2014-06-09 22:37:08] DEBUG: Building modules in 'native'...
[2014-06-09 22:37:09] DEBUG: FinMag          5057:1645a3f8da79e3da16e98f59b61f20290fecd34f
[2014-06-09 22:37:09] DEBUG: Dolfin          1.4.0                Matplotlib      1.3.1               
[2014-06-09 22:37:09] DEBUG: Numpy           1.8.1                Scipy           0.12.0              
[2014-06-09 22:37:09] DEBUG: IPython         2.1.0                Python          2.7.5+              
[2014-06-09 22:37:09] DEBUG: Paraview        4.0.1-1              Sundials        2.5.0               
[2014-06-09 22:37:09] DEBUG: Boost-Python    <unknown>            Linux           Linux Mint 16 Petra 
[2014-06-09 22:37:09] DEBUG: Registering debug signal handler. Press Ctrl-Z at any time to stop execution and jump into the debugger.


In [2]:
# IPYTHON_TEST_IGNORE_OUTPUT
print "Mayavi Version: " + mayavi.__version__
!avconv -version

Mayavi Version: 4.1.0
avconv version 0.8.10-6:0.8.10-0ubuntu0.13.10.1, Copyright (c) 2000-2013 the Libav developers
  built on Feb  6 2014 20:53:28 with gcc 4.8.1
avconv 0.8.10-6:0.8.10-0ubuntu0.13.10.1
libavutil    51. 22. 2 / 51. 22. 2
libavcodec   53. 35. 0 / 53. 35. 0
libavformat  53. 21. 1 / 53. 21. 1
libavdevice  53.  2. 0 / 53.  2. 0
libavfilter   2. 15. 0 /  2. 15. 0
libswscale    2.  1. 0 /  2.  1. 0
libpostproc  52.  0. 0 / 52.  0. 0


Firstly, lets create some simulation data, or skip it if the data is already there.

In [3]:
dataPath = "./tutorial_2_data/"
dataFile = "./tutorial_2_data/DMI.pvd"
DFile = df.File(dataFile)

# Create directory to store output data neatly.
if not os.path.exists(dataPath):
    os.makedirs(dataPath)

# Check to see if we've done this before to save time.
if not os.path.exists(dataFile):
    
    # Create simulation object with single skyrmion magnetisation.
    mesh = df.RectangleMesh(-100, -100, 100, 100, 50, 50)
    Ms = 1.567e5
    sim = finmag.Simulation(mesh, Ms, unit_length=1e-9)
    sim.initialise_skyrmions(skyrmionRadius=70)
        
    # Add energy objects to the simulation.
    sim.add(finmag.energies.DMI(2.466e-4))    
    sim.add(finmag.energies.Exchange(3.5322e-13))
    sim.add(finmag.energies.Zeeman(np.array([0., 0., 0.5 * Ms])))          
    energyD = sim.get_interaction("DMI")
    
    # Run simulation and save DMI every so often.       
    for zI in xrange(101):
        sim.run_until(1e-11 * zI)
        DFile << energyD.energy_density_function()

Now to start drawing. The first stages will be to become more familiar with the hierarchy of Mayavi. Firstly we will create an engine object, and create a scene object belonging to that engine. That scene will be used for the drawing operations in this notebook, though for advanced applications it is worth noting that an engine can have multiple scenes.

In [4]:
# The engine object we will use to interface with Mayavi. To initialise the engine without off-screen rendering, use an
# OffScreenEngine object instead.
engine = mayavi.api.Engine()
#engine = mayavi.api.OffScreenEngine(
engine.start()

# Create a scene object for this engine, and obtain it. This can be modified freely and safely later.
scene = engine.new_scene().scene

The engine doesn't require much configuration, though we do need to configure the scene. More specifically, we want to change the background colour of the scene, which can be done safely now.

In [5]:
# Set the background of the scene to black (RGB=000)
scene.background = (0., 0., 0.)

Now we load the data we generated into Mayavi. More specifically, we load the first VTU file of this relaxation and operate on that.

In [6]:
# Load the first VTU file we generated and define our scene using this.
vtkxml_file_reader = engine.open(dataPath + "DMI000000.vtu")

Now we can apply a surface module to represent our data and set a colourmap to use.

In [7]:
# Create a surface module object, and add it to our data read from file.
surface = mayavi.modules.surface.Surface()
engine.add_filter(surface, vtkxml_file_reader)

# Grab the colour manager.
colour_man = engine.scenes[0].children[0].children[0].scalar_lut_manager

# Change the colormap to something more friendly.
colour_man.lut_mode = 'PuOr'

We can add a colourbar using out colour manager also.

In [ ]:
# Show me the colourbar, resize it and move it.
colour_man.show_scalar_bar = True
colour_man.scalar_bar_representation.position = np.array([ 0.77,  0.03 ])
colour_man.scalar_bar_representation.position2 = np.array([ 0.049,  1.09 ])

# Show me the legend too, and set the range of values.
colour_man.show_legend = True
colour_man.label_text_property.color = (1.0, 1.0, 1.0)
colour_man.label_text_property.font_size = 40
colour_man.number_of_labels = 21

lLimit = -8e4
uLimit = 2e3
colour_man.data_range = np.array([lLimit, uLimit])

Another important aspect of our image is the camera position, which can be fully defined as below. Note that moving the camera requires a specific call to re-render the scene.

In [ ]:
height = 400.
scene.camera.position = [0, 0., height]
scene.camera.clipping_range = [height - 100, height + 100]
scene.camera.focal_point = [0., 0., 0.]
scene.camera.view_angle = 30.
scene.camera.view_up = [0., 1., 0.]

# Normally the scene is re-rendered when an object changes. Since we are not changing any filters or modules, the scene
# needs to be re-rendered manually.
scene.render()

Now that our scene is fully configured as we would like it, we can proceed to create our animation. In Mayavi, this can be achieved by loading the next VTU frame, saving a PNG file, and by using an external program to create the animation. The following example summarises this.

In [9]:
# IPYTHON_TEST_IGNORE_OUTPUT

# Iterate over each VTU file.
for zI in xrange(101):

    # Load the file corresponding to the next timestep. Mayavi is clever and knows that they are connected. This won't
    # destroy our pipeline since the engine will only modify this object and not the ones above it in the pipeline. However,
    # the range of values specified earlier will need to be restated before saving the image.
    vtkxml_file_reader.timestep = zI
    colour_man.data_range = np.array([lLimit, uLimit])
    scene.save_png(dataPath + "DMI{:06d}.png".format(zI))
    
# Create the animation (how exciting!)
!avconv -r 10 -y -i {dataPath}DMI%06d.png -same_quant {dataPath}out.mov -pass 2

avconv version 0.8.9-6:0.8.9-0ubuntu0.13.04.1, Copyright (c) 2000-2013 the Libav developers
  built on Nov  9 2013 19:09:48 with gcc 4.7.3
Input #0, image2, from './tutorial_2_data/DMI%06d.png':
  Duration: 00:00:10.10, start: 0.000000, bitrate: N/A
    Stream #0.0: Video: png, rgb24, 1920x1026, 10 fps, 10 tbr, 10 tbn, 10 tbc
Incompatible pixel format 'rgb24' for codec 'mpeg4', auto-selecting format 'yuv420p'
[buffer @ 0x1735b00] w:1920 h:1026 pixfmt:rgb24
[avsink @ 0x1735d20] auto-inserting filter 'auto-inserted scaler 0' between the filter 'src' and the filter 'out'
[scale @ 0x1720be0] w:1920 h:1026 fmt:rgb24 -> w:1920 h:1026 fmt:yuv420p flags:0x4
Output #0, mov, to './tutorial_2_data/out.mov':
  Metadata:
    encoder         : Lavf53.21.1
    Stream #0.0: Video: mpeg4, yuv420p, 1920x1026, q=2-31, 200 kb/s, 10 tbn, 10 tbc
Stream mapping:
  Stream #0:0 -> #0:0 (png -> mpeg4)
Press ctrl-c to stop encoding

video:2549kB audio:0kB global headers:0kB muxing overhead 0.060636%


You may want to experiment with more exotic modules and filters than I have here. There are many examples available on the mayavi documentation website that can be applied using a similar framework to this example. Some may find using a GUI more intuitive to explore the options that Mayavi offers; by using the Record button (the small red circle), it is possible to perform actions in the GUI while dumping actions to a Python script. This script can then be executed to recreate the workspace and pipeline, or it can be modified to manipulate files in a general case.

### References:

[1] http://docs.enthought.com/mayavi/mayavi/

[2] http://docs.enthought.com/mayavi/mayavi/overview.html#scenes-data-sources-and-visualization-modules-the-pipeline-model